In [1]:
import numpy as np
from collections import Counter

class BaggingClassifier:
    """
    Bagging ensemble for classification (with any base estimator).
    Uses majority vote for prediction.
    """
    def __init__(self, base_estimator, n_estimators=10, max_samples=1.0):
        """
        Parameters:
        -----------
        base_estimator : object
            A class implementing fit(X, y) and predict(X).
        n_estimators : int
            Number of base estimators in the ensemble.
        max_samples : float
            Fraction of samples to draw from X to train each base estimator.
        """
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.estimators = []

    def fit(self, X, y):
        n_samples = X.shape[0]
        n_subsamples = int(self.max_samples * n_samples)
        self.estimators = []
        for _ in range(self.n_estimators):
            indices = np.random.choice(n_samples, n_subsamples, replace=True)
            X_sub, y_sub = X[indices], y[indices]
            estimator = self.base_estimator()
            estimator.fit(X_sub, y_sub)
            self.estimators.append(estimator)

    def predict(self, X):
        # Collect predictions from all estimators and use majority vote
        preds = np.array([estimator.predict(X) for estimator in self.estimators])
        # Mode along axis=0 (samples)
        majority = [Counter(preds[:, i]).most_common(1)[0][0] for i in range(X.shape[0])]
        return np.array(majority)

class BaggingRegressor:
    """
    Bagging ensemble for regression (with any base estimator).
    Uses mean for prediction.
    """
    def __init__(self, base_estimator, n_estimators=10, max_samples=1.0):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.estimators = []

    def fit(self, X, y):
        n_samples = X.shape[0]
        n_subsamples = int(self.max_samples * n_samples)
        self.estimators = []
        for _ in range(self.n_estimators):
            indices = np.random.choice(n_samples, n_subsamples, replace=True)
            X_sub, y_sub = X[indices], y[indices]
            estimator = self.base_estimator()
            estimator.fit(X_sub, y_sub)
            self.estimators.append(estimator)

    def predict(self, X):
        # Collect predictions from all estimators and use mean
        preds = np.array([estimator.predict(X) for estimator in self.estimators])
        return np.mean(preds, axis=0)

# Example usage:
if __name__ == "__main__":
    # Dummy base estimator for demonstration
    class DummyClassifier:
        def fit(self, X, y):
            self.majority = Counter(y).most_common(1)[0][0]
        def predict(self, X):
            return np.full(X.shape[0], self.majority)

    class DummyRegressor:
        def fit(self, X, y):
            self.mean = np.mean(y)
        def predict(self, X):
            return np.full(X.shape[0], self.mean)

    # Classification example
    Xc = np.array([[1,2],[2,1],[1,1],[2,2],[3,3],[3,2]])
    yc = np.array([0,0,0,1,1,1])
    bag_clf = BaggingClassifier(base_estimator=DummyClassifier, n_estimators=5)
    bag_clf.fit(Xc, yc)
    print("BaggingClassifier predictions:", bag_clf.predict(np.array([[2,2],[1,1],[3,3]])))

    # Regression example
    Xr = np.array([[1],[2],[3],[4],[5],[6]])
    yr = np.array([1.0, 1.5, 2.0, 4.0, 5.0, 6.0])
    bag_reg = BaggingRegressor(base_estimator=DummyRegressor, n_estimators=5)
    bag_reg.fit(Xr, yr)
    print("BaggingRegressor predictions:", bag_reg.predict(np.array([[2],[4],[6]])))

BaggingClassifier predictions: [0 0 0]
BaggingRegressor predictions: [2.9 2.9 2.9]
